# NAS with auto-pytorch
Modelle des Ensembles anzeigen

In [30]:
!pip install --upgrade xlrd # important to upgrade to open xls file
!pip install autoPyTorch 

# import needed packages
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.ensemble
import matplotlib.pyplot as plt
import numpy as np
import time

In [33]:
# Generate Train and Test Data
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/AutoML/Concrete_Data.xls', engine='xlrd')
X = data.iloc[:, 0:8].to_numpy()
y = data.iloc[:, 8].to_numpy()

print('Size of X: ', X.shape)

X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y,random_state=1)

Size of X:  (1030, 8)


In [34]:
# Import TabularRegressionTask from autoPyTorch
from autoPyTorch.api.tabular_regression import TabularRegressionTask

api = TabularRegressionTask()

# .search() takes train and test data, asks for optimization metric and how long the algorithm should run.
# set memory_limit=None to use complete memory instead of default 4096 MB, so that used algorithms dont crash.
api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test.copy(),
    y_test=y_test.copy(),
    optimize_metric='r2',
    total_walltime_limit=1000,
    func_eval_time_limit_secs=50,
    memory_limit=None,
    enable_traditional_pipeline=False # just NNs and not kNN and traditional stuff...
)

[WARNING] [2022-05-05 12:14:56,231:Client-Validation] AutoPyTorch previously received features of type <class 'numpy.ndarray'> yet the current features have type <class 'pandas.core.frame.DataFrame'>. Changing the dtype of inputs to an estimator might cause problems
[WARNING] [2022-05-05 12:14:56,258:Client-Validation] AutoPyTorch previously received features of type <class 'numpy.ndarray'> yet the current features have type <class 'pandas.core.frame.DataFrame'>. Changing the dtype of inputs to an estimator might cause problems


/usr/local/lib/python3.7/dist-packages/smac/intensification/parallel_scheduling.py:152: UserWarning: Hyperband is intended to be used with more than 1 worker but num_workers=1
  num_workers
/usr/local/lib/python3.7/dist-packages/smac/intensification/parallel_scheduling.py:152: UserWarning: Hyperband is intended to be used with more than 1 worker but num_workers=1
  num_workers
/usr/local/lib/python3.7/dist-packages/smac/intensification/parallel_scheduling.py:152: UserWarning: Hyperband is intended to be used with more than 1 worker but num_workers=1
  num_workers
/usr/local/lib/python3.7/dist-packages/smac/intensification/parallel_scheduling.py:152: UserWarning: Hyperband is intended to be used with more than 1 worker but num_workers=1
  num_workers
/usr/local/lib/python3.7/dist-packages/smac/intensification/parallel_scheduling.py:152: UserWarning: Hyperband is intended to be used with more than 1 worker but num_workers=1
  num_workers
/usr/local/lib/python3.7/dist-packages/smac/intens

In [35]:
y_pred = api.predict(X_test)

# Rescale the Neural Network predictions into the original target range
score = api.score(y_pred, y_test)

print(score)

# Print the final ensemble built by AutoPyTorch
print(api.show_models())

# Print statistics from search
print(api.sprint_statistics())

# wie wird der validation score gemessen?

{'r2': 0.8819549312612294}
|    | Preprocessing                                                 | Estimator                                                       |   Weight |
|---:|:--------------------------------------------------------------|:----------------------------------------------------------------|---------:|
|  0 | SimpleImputer,NoEncoder,StandardScaler,NoFeaturePreprocessing | no embedding,ShapedMLPBackbone,FullyConnectedHead,nn.Sequential |     0.92 |
|  1 | SimpleImputer,NoEncoder,Normalizer,PolynomialFeatures         | no embedding,MLPBackbone,FullyConnectedHead,nn.Sequential       |     0.06 |
|  2 | SimpleImputer,NoEncoder,StandardScaler,NoFeaturePreprocessing | no embedding,ShapedMLPBackbone,FullyConnectedHead,nn.Sequential |     0.02 |
autoPyTorch results:
	Dataset name: f9234d5e-cc6c-11ec-81bd-0242ac1c0002
	Optimisation Metric: r2
	Best validation score: 0.9090103249648109
	Number of target algorithm runs: 53
	Number of successful target algorithm runs: 48
	Number

In [40]:
print(api.models_)

{(1, 12, 50.0): ________________________________________
	TabularRegressionPipeline
________________________________________
0-) imputer: 
	SimpleImputer

1-) encoder: 
	NoEncoder

2-) scaler: 
	Normalizer

3-) feature_preprocessor: 
	PolynomialFeatures

4-) tabular_transformer: 
	TabularColumnTransformer

5-) preprocessing: 
	EarlyPreprocessing

6-) network_embedding: 
	autoPyTorch.pipeline NoEmbedding

7-) network_backbone: 
	autoPyTorch.pipeline MLPBackbone

8-) network_head: 
	autoPyTorch.pipeline FullyConnectedHead

9-) network: 
	Sequential ({'random_state': RandomState(MT19937) at 0x7F1BE486DAF0, '_fit_requirements': [FitRequirement(name='network_head', supported_types=(<class 'torch.nn.modules.module.Module'>,), user_defined=False, dataset_property=False), FitRequirement(name='network_backbone', supported_types=(<class 'torch.nn.modules.module.Module'>,), user_defined=False, dataset_property=False), FitRequirement(name='network_embedding', supported_types=(<class 'torch.nn.modu

# Auto-sklearn
Switching to auto-sklearn. AutoPyTorch is based on auto-sklearn so using auto-sklearn might reveal more about AutoPyTorch Functionalities since the Documentation is poor.

In [14]:
# Generate Train and Test Data
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/AutoML/Concrete_Data.xls', engine='xlrd')
X = data.iloc[:, 0:8].to_numpy()
y = data.iloc[:, 8].to_numpy()

print('Size of X: ', X.shape)

X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y,random_state=1)

Size of X:  (1030, 8)
